# Spark Cluster Test
Testing parallel processing with multiple workers

In [8]:
from pyspark.sql import SparkSession
import time

In [9]:
spark = SparkSession.builder \
    .appName("NotebookTest") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

In [10]:
print(f"Spark {spark.version}")
print(f"Master: {spark.sparkContext.master}")

Spark 3.5.0
Master: spark://spark-master:7077


In [11]:
import os 
path= '/opt/spark/work-dir/data/sample.txt'
print(os.path.exists('/opt/spark/work-dir/data/sample.txt'))

True


In [12]:
text_df = spark.read.text(path)
print(f"Lines: {text_df.count()}")
text_df.show(5, truncate=False)

Lines: 30
+---------------------------------------------------------------------------+
|value                                                                      |
+---------------------------------------------------------------------------+
|Sahar works with distributed data processing systems every day.            |
|Kshitij manages the Spark cluster configuration and deployment.            |
|Rabina develops machine learning pipelines using PySpark.                  |
|                                                                           |
|Apache Spark is a unified analytics engine for large-scale data processing.|
+---------------------------------------------------------------------------+
only showing top 5 rows



In [13]:
data = [("Sahar", 24, "Data Engineer"),
        ("Kshitij", 27, "DevOps Engineer"),
        ("Rabina", 29, "ML Engineer"),
        ("Manu", 26, "Backend Developer"),
        ("Ola", 29, "Frontend Developer")]

test_df = spark.createDataFrame(data, ["name", "age", "role"])
test_df.show()

Py4JJavaError: An error occurred while calling o112.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 4 times, most recent failure: Lost task 0.3 in stage 9.0 (TID 13) (172.19.0.6 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 8) than that in driver 3.11, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunne

In [ ]:
df.groupBy("role").count().show()

In [ ]:
large_range = spark.range(0, 1000000)
result = large_range.filter(large_range.id % 2 == 0).count()
print(f"Even numbers: {result}")

In [ ]:
rdd = spark.sparkContext.parallelize(range(100))
squared = rdd.map(lambda x: x * x).collect()
print(f"Sum of squares (0-99): {sum(squared)}")

In [ ]:
spark.stop()